# Working interface models

In [ ]:
%matplotlib widget
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

# FE 2D discretization

In [ ]:
n_y_e = 20
n_inner_x_e = 2
n_outer_x_e = 2
L_x = 1.0
P = 1.0
R_in = 10 / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS2DElastic(E=37000, nu=0.18)
m2 = MATS2DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor = P,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [ ]:
material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3, 
     E_N=1, S_N=0.5, c_N = 3, m = 0.05, f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [ ]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [ ]:
xd12.I.dofs, xd12.J.dofs

## Sliding dc

In [ ]:
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=16)
bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

## Sliding fc

In [ ]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 4 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [ ]:
m.bc=bc1
s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.05
s.tstep.fe_domain.serialized_subdomains

In [ ]:
xd12.hidden = True
s.reset()
s.run()

In [ ]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [ ]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(10,4))
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Nx[:,0,0].flatten(), color='red', label='normal damage')
ax3.plot(time, omega_Tx[:,0,0].flatten(), color='green', label='tangential damage')

ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_Tx[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()
f.tight_layout()